In [ ]:
import sys
sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    get_load_config_from_yaml,
)
from importlib import reload
import yaml
import core
import keras
import core.keras_models.RegressionTransformer as RegressionTransformer
import numpy as np

PLOTS_DIR = f"plots/regresion_transformer/"
MODEL_DIR = f"models/regresion_transformer/"
CONFIG_PATH = "../config/workspace_config.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


data_config = DataProcessor.load_from_npz(
    load_config["nominal"], max_events=8_000_000, event_numbers="even"
)

X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)
del DataProcessor # Free memory

2026-01-06 10:55:44.971418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767693344.993647 3891938 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767693345.000981 3891938 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767693345.019091 3891938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767693345.019110 3891938 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767693345.019112 3891938 computation_placer.cc:177] computation placer alr

In [2]:
reload(RegressionTransformer)
Transformer = RegressionTransformer.FeatureConcatTransformer(data_config)

In [3]:
Transformer.build_model(
    hidden_dim=128,
    num_heads=8,
    num_layers=12,
    dropout_rate=0.1,
    compute_HLF=False,
    log_variables=True,
)

Transformer.adapt_normalization_layers(X_train)

Transformer.compile_model(
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4,clipnorm=1.0),
    loss={
        "normalized_regression": core.utils.MagnitudeDirectionLoss(),
        "assignment": core.utils.AssignmentLoss(),
    },
    metrics={
        "normalized_regression": core.utils.RegressionDeviation(),
        "assignment": core.utils.AssignmentAccuracy(),
    },
    loss_weights={
        "normalized_regression": 1.0,
        "assignment": 1.0,
    },
)


I0000 00:00:1767693492.997863 3891938 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0
I0000 00:00:1767693497.873257 3894486 service.cc:152] XLA service 0x7f3d34004280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767693497.873281 3894486 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-01-06 10:58:17.885584: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1767693497.899207 3894486 cuda_dnn.cc:529] Loaded cuDNN version 91500
I0000 00:00:1767693498.049407 3894486 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Adapted normalization layer:  lep_input_normalization
Adapted normalization layer:  jet_input_normalization


2026-01-06 10:58:31.849374: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-06 10:58:35.173446: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator pre_met_input_normalization_model_1/met_input_transform_1/assert_equal_1/Assert/Assert


Adapted normalization layer:  met_input_normalization


In [4]:
Transformer.model.summary()

Model: "keras_model_wrapper"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_inputs          │ (None, 6, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 6)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 2)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 7)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 6)      │         13 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 7)      │         15 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ConcatLeptonCharge  │ (None, 2, 6)      │          0 │ lep_input_normal… │
│ (ConcatLeptonCharg… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_embedding_mlp   │ (None, 6, 128)    │     10,313 │ jet_input_normal… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lepton_embedding_m… │ (None, 2, 128)    │      9,767 │ ConcatLeptonChar… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_embedding_mlp   │ (None, 1, 128)    │      6,764 │ met_input_normal… │
│ (MLP)               │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 9, 128)    │          0 │ jet_embedding_ml… │
│ (Concatenate)       │                   │            │ lepton_embedding… │
│                     │                   │            │ met_embedding_ml… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expand_jet_mask     │ (None, 9)         │          0 │ jet_mask[0][0]    │
│ (ExpandJetMask)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 2,076,433 (7.92 MB)

 Trainable params: 2,076,400 (7.92 MB)

 Non-trainable params: 33 (144.00 B)

In [5]:
Transformer.train_model(
    X_train,
    y_train,
    batch_size=1024,
    epochs=10,
    copy_data=True,
)

Epoch 1/50


2026-01-06 10:59:23.144538: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1/met_input_transform_1/assert_equal_1/Assert/Assert
2026-01-06 11:00:23.736869: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'input_reduce_slice_fusion_1', 44 bytes spill stores, 44 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_convert_xor_fusion', 732 bytes spill stores, 732 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_convert_xor_fusion_1', 736 bytes spill stores, 736 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_convert_xor_fusion_2', 736 bytes spill stores, 736 bytes spill loads



 632/5549 ━━━━━━━━━━━━━━━━━━━━ 7:44 94ms/step - assignment_accuracy: 0.1500 - assignment_loss: 0.1735 - loss: 3.1799 - normalized_regression_deviation: 0.7353 - normalized_regression_loss: 3.0064

KeyboardInterrupt: 

In [ ]:
Transformer.save_model(MODEL_DIR + "model.keras")

In [ ]:
pred = Transformer.trainable_model.predict_dict(
    {
        "jet_inputs": X_val["jet"],
        "lep_inputs": X_val["lepton"],
        "met_inputs": X_val["met"],
    }

)

In [ ]:
neutrinos = Transformer.reconstruct_neutrinos(X_val)

In [ ]:
np.abs((neutrinos - y_val["neutrino_truth"])/y_val["neutrino_truth"]).mean(),np.abs((X_val["nu_flows_neutrino_truth"] - y_val["neutrino_truth"])/y_val["neutrino_truth"]).mean()

In [ ]:
X_val["nu_flows_neutrino_truth"]

In [ ]:
y_val["neutrino_truth"]

In [ ]:
neutrinos